In [55]:
from io import open
import unicodedata
import re
import random
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from scipy.spatial.distance import cosine
import warnings
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.stem.snowball import SnowballStemmer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
warnings.filterwarnings("ignore")

In [2]:
# Определим по умолчанию 2 токена которые будут нам информировать о начале предложения и конце предложения (SOS и EOS):
SOS_token = 0
EOS_token = 1

# Создадим объект словаря нашего языка, который будет хранить данные по маппингу слов - index2word и обратно word2index и плюс второстепенные методы по добавлению токена и обработке предложений:
class LanguageVocabulary(object):
    def __init__(self, name):
        # название языка
        self.name = name
        # словарик word2index который хранит соответственно кодировку слова в целочисленный индекс словаря
        self.word2index = {}
        # обычный словарик который хранит распределение слов, сколько слов мы использовали и сколько обнаружили
        self.word2count = {}
        # Обратный словарик словарю word2index где хранятся уже индексы и замаппенные слова к каждому индексу, нужен будет для расшифровки последовательности
        self.index2word = {0: "SOS", 1: "EOS"}
        # Count SOS and EOS, храним просто общее количество слов в нашем словаре, то есть количество токенов в сформированном словарике нашего языка
        self.n_words = 2
        self.MAX_LEN=0 #самое длинное предложение

    def add_sentence(self, sentence):
        """
        Метод класса, для добавления предложения в словарь.
        Каждое предложение поступающее к нам, будет разбираться на
        примитивные токены и добавляться в словарь при помощи метода класса addword()
        """
        length=len(sentence.split(' '))
        if length>self.MAX_LEN:
            self.MAX_LEN=length
        for word in sentence.split(' '):
            self.add_word(word)


    def add_word(self, word):
        # проверяем не входит ли наше слово в словарь word2index
        if word not in self.word2index:
            # добавляем в качестве ключа слово а в качестве значения последнее n_words
            self.word2index[word] = self.n_words
            # меняем на единичку
            self.word2count[word] = 1
            # и соответственно меняем и index2word словарик добавляя уже слово для декодирования
            self.index2word[self.n_words] = word
            # инкрементируем n_words
            self.n_words += 1
        else:
            # Если такое уже слово есть просто добавляем 1 что добавилось одно слово
            self.word2count[word] += 1

In [3]:
# Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427
def unicode_to_ascii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalize_string(s):
    # Декодируем из юникода в ascii
    s = unicode_to_ascii(s.lower().strip())
    # Что означает данное регулярное выражение - точку, !, ? меняем на пробел чтобы этот символ стоял отдельно от всех
    # https://docs.python.org/3/library/re.html - стандартная (родная) библиотка Python которая нужна для работы с регулярными выражениями
    s = re.sub(r"([.!?])", r" \1", s)
    # оставляем только наборы символов указанных в паттерне регулярного выражения остальное заменим на пробел
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [4]:
def read_languages(lang1, lang2, path,  reverse=False):
    print("Reading lines...")
    # Берем документ корпуса, лежащий в директории ./data/___.txt подставляя значения указанных языков в нашем случае eng-fra, он читается бьется на предложения
    lines = open(path+"/"+"%s-%s.txt" % (lang1, lang2), encoding='utf-8').read().strip().split('\n')
    #print(lines)
    # Разбиваем построчно и нормализуем строку:
    pairs = [[normalize_string(s) for s in l.split('\t')] for l in lines]
    # Можем создавать и проходить как с целевого языка на исходный так и наоборот:
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = LanguageVocabulary(lang2)
        output_lang = LanguageVocabulary(lang1)
    else:
        input_lang = LanguageVocabulary(lang1)
        output_lang = LanguageVocabulary(lang2)
    return input_lang, output_lang, pairs

In [7]:
def make_file(lang1, lang2, path, file_name):
    f1= open(path+"/"+file_name, mode='r', encoding='utf-8')
    f2=open(path+"/%s-%s.txt" % (lang1,lang2), mode='w', encoding='utf-8')

    lines=f1.readlines()
    pairs=[re.split(r"[\t]", line)[:2] for line in lines]



    for i in range(len(pairs)):
        f2.write(pairs[i][0]+"\t"+pairs[i][1]+"\n")


    f1.close()
    f2.close()

In [58]:
make_file("eng","ita", "sample_data/translation_data", "ita.txt")

In [59]:
def prepare_data(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = read_languages(lang1, lang2, "sample_data/translation_data", reverse)
    print("Read %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.add_sentence(pair[0])
        output_lang.add_sentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    print("Max len in {}: {}".format(input_lang.name, input_lang.MAX_LEN))
    print("Max len in {}: {}".format(output_lang.name, output_lang.MAX_LEN))
    MAX_LEN=input_lang.MAX_LEN if input_lang.MAX_LEN>output_lang.MAX_LEN else output_lang.MAX_LEN
    return input_lang, output_lang, pairs, MAX_LEN

In [ ]:
input_lang, output_lang, pairs, MAX_LEN = prepare_data('eng', 'ita', False)
print(random.choice(pairs))
print(MAX_LEN)

# Encoder

In [11]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        # Как помним hidden_size - размер скрытого состояния
        self.hidden_size = hidden_size
        # Слой Эмбеддингов, который из входного вектора последовательности (либо батча) отдаст представление последовательности для скрытого состояния
        # FYI: в качестве Input_size у нас размер словаря
        self.embedding = nn.Embedding(input_size, hidden_size)
        # И соответственно рекуррентная ячейка GRU которая принимает MxM (hidden на hidden)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        # Приводим эмбеддинг к формату одного предлоежния 1х1 и любая размерность
        embedded = self.embedding(input).view(1, 1, -1)
        # Нужно для следующего шага пока не запутываемся :) просто присвоили наш эмбеддинг
        output = embedded
        # и соответственно подаем все в ГРЮ ячейку (эмбеддинг и скрытые состояния)
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        # Дополнительно сделаем инициализацию скрытого представления (просто заполним нулями)
        return torch.zeros(1, 1, self.hidden_size, device=device)

# Decoder

In [12]:
class AttentionDecoder(nn.Module):
    # Будьте внимательны, теперь на вход мы получаем размер скрытого представления
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LEN):
        super(AttentionDecoder, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))  #вектор контекста c

        output = torch.cat((embedded[0], attn_applied[0]), 1)  #[y, c]
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [13]:
# Токены кодируем в целочисленное представление
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


# Берем предложение с указанным языком, делаем из него индексы и вставляем метку конца предложения, превращаем в тензор:
def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

# Для создания тензора из пар:
def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [14]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LEN):
    # Просто инициализируем скрытое представление для энкодера
    encoder_hidden = encoder.initHidden()
    # Скиыдваем градиенты для алгоритма градиентного спуска как и у энкодера так и у дэкодера
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    # Получаем размер в словаря (токенов) для входящего и выходящего тензора так как мы пробегаемся по каждому предложению по кусочкам
    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)
    # Создаем переменную где будем хранить наши выходы из энкодера (в данной реализации пока не юзаем, далее будет еще один вариант)
    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
    loss = 0
    # пробегаем по длине входящего тензора и в экодер передаем последовательно каждый из токенов:
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        # Сохраняем все выходы из энкодера для одного слова (для передачи в декодер, ниже)
        encoder_outputs[ei] = encoder_output[0, 0]


    # Закончили с энкодером пошли к декодеру, как было сказано декодер начинается с SOS
    decoder_input = torch.tensor([[SOS_token]], device=device)
    # FYI здесь мы скрытое представление из энкодера передаем в скрытое представление в декодер, то есть после знака =
    # у нас будут ходить градиенты из декодера в энкодер, то есть когда мы будем считать градиенты, они сначала пробегут по декодеру
    # дойдут до знака = перескочат в энкодер и будут дальше считаться по энкодеру и эти градиенты сохранятся в соответствующих тензорах
    # и когда будут отрабатывать разные оптимайзеры (у нас их 2) у них будут соответствующие правильные градиенты которые смогут правильно отработать
    decoder_hidden = encoder_hidden

    # Будем использовать Teacher Forcing в части случае (подставляя правильную последовательность)
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    if use_teacher_forcing:
        # Подаем decoder_input = torch.tensor([[SOS_token]], device=device) то есть по одному слову и скрытое представление
        for di in range(target_length):
            # Переведенное предложение и скрытое представление
            decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
            # Считаем ошибку
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing
    else:
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input
            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break
    loss.backward()
    encoder_optimizer.step()
    decoder_optimizer.step()
    return loss.item() / target_length

In [15]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / percent
    rs = es - s
    return '%s (- eta: %s)' % (asMinutes(s), asMinutes(rs))

In [62]:
def trainIters(encoder, decoder, n_iters, print_every=1000, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    # Делаем выборку наших пар функцией которую создали до
    training_pairs = [tensorsFromPair(random.choice(pairs)) for i in range(n_iters)]
    # FYI! Используем Negative Log-Likelihood Loss потому что log softmax уже присутствует в модели
    criterion = nn.NLLLoss()

    for epoch in range(1, n_iters + 1):
        training_pair = training_pairs[epoch - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]
        # Используем функцию для тренировки на отдельных токенах, которую написали выше
        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        
        if epoch % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, epoch / n_iters),
                                         epoch, epoch / n_iters * 100, print_loss_avg))

        


In [19]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LEN):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] = encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [100]:
#для оценки перевода

def calc_accuracy(y, target, output_lang, lang_name, hidden_size):
    #y - полученный перевод
    #target - целевой перевод
    Stemmer=SnowballStemmer(lang_name)
    y_normalized=[]
    target_normalized=[]
    for token in y.split(' '):
        y_normalized.append(Stemmer.stem(token))
    y_normalized=' '.join(y_normalized)  
    for token in target.split(' '):
      target_normalized.append(Stemmer.stem(token))
    target_normalized=' '.join(target_normalized)  

    corpus=[y_normalized, target_normalized]
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(corpus).toarray()
    y_vectorized=X[0]
    target_vectorized=X[1]
    
    return 1-cosine(y_vectorized, target_vectorized)

In [98]:
def evaluateRandomly(encoder, decoder, output_lang, lang_name, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words[:-1]) #без <EOS>
        print('<', output_sentence)
        print("accuracy: {:.4f}". format(calc_accuracy(output_sentence, pair[1], output_lang, lang_name, hidden_size)))
        print('\n')

# Обучение

In [ ]:
hidden_size = 512
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder1 = AttentionDecoder(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)
trainIters(encoder1, decoder1, 10000, print_every=1000)

In [102]:
evaluateRandomly(encoder1, decoder1, output_lang, "italian")

> stay .
= resti .
< resti .
accuracy: 1.0000


> be calm .
= sii calma .
< sia calma .
accuracy: 0.5000


> smile .
= sorridi .
< sorridete .
accuracy: 1.0000


> burn it .
= brucialo .
< lo bruci .
accuracy: 0.0000


> go on .
= andate avanti .
< continua .
accuracy: 0.0000


> i fell .
= io sono caduto .
< sono caduto .
accuracy: 0.8165


> burn it .
= bruciatela .
< lo bruci .
accuracy: 0.0000


> wait !
= aspetti !
< aspetta !
accuracy: 1.0000


> i fled .
= fuggii .
< sono fuggita .
accuracy: 0.7071


> burn it .
= bruciala .
< lo bruci .
accuracy: 0.0000


